<a href="https://colab.research.google.com/github/raghunadh-eva/eva6/blob/master/EVA6_S4_ASSIGNMENT/PART2/EVA4_S4_ASSIGNMENT_PART2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        #28x28x32
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        #28x28x32
        self.pool1 = nn.MaxPool2d(2, 2)
        #14x14x32
        self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
        #14x14x32
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        #14x14x32
        self.pool2 = nn.MaxPool2d(2, 2)
        #7x7x32
        self.conv5 = nn.Conv2d(32, 32, 3)
        #5x5x32
        self.conv6 = nn.Conv2d(32, 32, 3)
        #3x3x32
        self.conv7 = nn.Conv2d(32, 10, 3)
        #1x1x32

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        #print(x.shape)
        #print(x)
        x = x.view(-1, 10)
        #print(x.shape)
        #print(x)
        return F.log_softmax(x)

In [3]:
class NewNetwork(nn.Module):
    def __init__(self):
        super(NewNetwork, self).__init__()
        
        #Using the concepts of Block
        #Block1 - Hit receptive field of 5x5
        self.conv1 = nn.Sequential(
            #28x28x1 | 3x3x1x32 -> 28x28x32
            #28x28x32 | 3x3x32 -> 28x28x32
            #28x28x32 | 26x26x32
            #batch normalization --> amplify the features
            #maxpool2sd --> 12x12x32
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2)
            #nn.Dropout2d(0.05)
            ##Dont use dropoout in first pass
            #nn.Dropout()
        )
        
        self.conv2 = nn.Sequential(
            #14x14x32 | 3x3x1x32 -> 14x14x32
            #batch normalization --> amplify the features
            #maxpool2sd --> 12x12x32
            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2)
            #nn.Dropout2d(0.05)
        )
        #7x7x32 --> #7x7x32
        #Global average pooling
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 20, 1),
            nn.ReLU(),
            nn.AvgPool2d(7)
        )
        #7x7x10
        self.fc = nn.Sequential(
            nn.Linear(20, 10)
        )
                
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 32, 28, 28]           9,248
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]           9,248
            Conv2d-5           [-1, 32, 14, 14]           9,248
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Conv2d-7             [-1, 32, 5, 5]           9,248
            Conv2d-8             [-1, 32, 3, 3]           9,248
            Conv2d-9             [-1, 10, 1, 1]           2,890
Total params: 49,450
Trainable params: 49,450
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.55
Params size (MB): 0.19
Estimated Total Size (MB): 0.74
---------------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


#New parameter counts

In [5]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = NewNetwork().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
       BatchNorm2d-3           [-1, 32, 28, 28]              64
            Conv2d-4           [-1, 32, 28, 28]           9,248
              ReLU-5           [-1, 32, 28, 28]               0
       BatchNorm2d-6           [-1, 32, 28, 28]              64
         MaxPool2d-7           [-1, 32, 14, 14]               0
            Conv2d-8           [-1, 32, 14, 14]           9,248
              ReLU-9           [-1, 32, 14, 14]               0
      BatchNorm2d-10           [-1, 32, 14, 14]              64
        MaxPool2d-11             [-1, 32, 7, 7]               0
           Conv2d-12             [-1, 20, 7, 7]             660
             ReLU-13             [-1, 20, 7, 7]               0
        AvgPool2d-14             [-1, 2

In [6]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data1', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data1', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [7]:
import numpy as np
import matplotlib.pyplot as plt
#batch = next(iter(train_loader))



In [8]:
#images, labels = batch
#grid = torchvision.utils.make_grid(images, nrow=10)
#plt.figure(figsize=(15,15))
#plt.imshow(np.transpose(grid, (1,2,0)))
#print('labels:', labels)

In [9]:
from tqdm.notebook import tqdm
def train(model, device, train_loader, optimizer, epoch):
    print('In epoch {}'.format(epoch))
    model.train()
    pbar = tqdm(train_loader)
    train_loss = 0
    train_correct = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
    #for data, target in train_loader:
    #    data, target = data.to(device), target.to(device)
    #    output = model(data)
    #    train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
    #    pred_train = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    #    train_correct += pred_train.eq(target.view_as(pred_train)).sum().item()
    #    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
    #    train_loss, train_correct, len(train_loader.dataset),
    #   100. * train_correct / len(train_loader.dataset)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:

model = NewNetwork().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=0.9)
#train(model, device, train_loader, optimizer, 1)
#model.train()
#pbar = tqdm(train_loader)
#enumerate(pbar)
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

In epoch 1




Test set: Average loss: 0.1176, Accuracy: 9659/10000 (96.6%)

In epoch 2




Test set: Average loss: 0.0864, Accuracy: 9756/10000 (97.6%)

In epoch 3




Test set: Average loss: 0.0684, Accuracy: 9789/10000 (97.9%)

In epoch 4




Test set: Average loss: 0.0785, Accuracy: 9737/10000 (97.4%)

In epoch 5


...



Test set: Average loss: 0.0510, Accuracy: 9850/10000 (98.5%)

In epoch 16




Test set: Average loss: 0.0513, Accuracy: 9876/10000 (98.8%)

In epoch 17




Test set: Average loss: 0.0584, Accuracy: 9837/10000 (98.4%)

In epoch 18




Test set: Average loss: 0.0605, Accuracy: 9833/10000 (98.3%)

In epoch 19




Test set: Average loss: 0.0561, Accuracy: 9839/10000 (98.4%)

